# Ajouter une explication de votre méthode ici

# PS: Il y a des fonctions utiles dans /src/util.py et src/plots.py
ds = read_csv(csvfile) - read csv file, and return as dict type

write(csvfile,mydict) - write a dict type data in csv file

In [2]:
%load_ext autoreload
%autoreload 2
import os 

import numpy as np
import matplotlib.pyplot as plt
import sys

current_dir = os.getcwd()

sys.path.append(os.path.abspath(os.path.join(current_dir, '..')))
from src import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# load s2 rad and xyz data
ds_xyz = read_csv(os.path.join(current_dir,'..','Simulation','xyz_ECOSTRESS.csv'))
ds_rho_s2a = read_csv(os.path.join(current_dir,'..','Simulation','rho_ECOSTRESS_S2A.csv'))
ds_rho_s2b = read_csv(os.path.join(current_dir,'..','Simulation','rho_ECOSTRESS_S2B.csv'))
ds_rho_s2 = [ds_rho_s2a,ds_rho_s2b]
s2_bnames = ['B4','B3','B2']
xyz_names = ['X','Y','Z']
sensor_names = ['S2A','S2B']

# load train fname list
f = open(os.path.join(current_dir,'..','Simulation','fnames_train.txt'),"r")
fnames_test = f.read().splitlines()
f.close()

def get_inds(fnames,fnames_sub):
    inds = []
    for fname in fnames_sub:
        inds.append(fnames.index(fname))
    return inds
inds = get_inds(ds_xyz['NAME'],fnames_test)

def build_sub_dataset(ds,inds):
    mydict = {}
    for key in list(ds.keys()):
        mydict[key] = np.array(ds[key])[inds]
    return mydict

ds_rho_s2_sub = []
for ds in ds_rho_s2:
    ds_rho_s2_sub.append(build_sub_dataset(ds,inds))

ds_xyz_sub = build_sub_dataset(ds_xyz,inds)

In [15]:
def moindre_carre(B,A):
    # Karine Ye and Anthony Evrard 
    P = np.eye(len(A))
    N = (A.T)@P@A
    K = (A.T)@P@B
    X = np.linalg.inv(N)@K
    V = B - A @ X
    n = len(A)
    p = len(A[0])
    sigma_2 = V.T@P@V/(n-p)
    Q = (sigma_2)*(np.linalg.inv(A.T@P@A))
    return X, Q, sigma_2, V

def inversion_d65(ds_s2,ds_xyz,matrix):
    # B = A X with B[mxkxs,1], A1[mxkxs, n] X[n,1], m is the number of samples, n is number of band, k is number of XYZ, s is number of sensor
    s2_bnames = ['B4','B3','B2']
    xyz_names = ['X','Y','Z']
    s = len(ds_s2)
    m = len(ds_s2[0]['B4'])
    n = len(s2_bnames)
    k = len(xyz_names)
    A = np.empty([m*k*s,n],dtype=float)
    B = np.empty([m*k*s,1],dtype=float)
    
    cpt = 0
    for ss in range(s):
        for kk in range(k):
            for mm in range(m):
                B[cpt,0] = ds_xyz[xyz_names[kk]][mm]
                for nn in range(n):
                    A[cpt,nn] = ds_s2[ss][s2_bnames[nn]][mm]*matrix[nn,kk]
                cpt += 1


    X, Q, sigma_2, V = moindre_carre(B,A)
    B1 =  A@X
    print('Fit RMSE =',np.sqrt(np.mean((B.flatten()-B1.flatten())**2)))
    
    new_matrix = matrix.copy()
    for kk in range(k):
        for nn in range(n):
            new_matrix[nn,kk] = matrix[nn,kk]*X[nn]
    return X,A,B,B1,new_matrix


X_out,A,B,B1,new_matrix = inversion_d65(ds_rho_s2_sub,ds_xyz_sub,matrix)
print('Estimated d65 (B4,B3,B2)')
print(X_out.flatten())
print('Modified Matrix')
print(new_matrix)


Fit RMSE = 0.009040559569658284
Estimated d65 (B4,B3,B2)
[0.80450672 1.02140103 1.08628878]
Modified Matrix
[[ 0.29847199  0.13998417 -0.03459379]
 [ 0.36974717  0.59956241 -0.43613824]
 [ 0.29112539  0.26070931  1.58924048]]
